In [283]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import copy
import json
from collections import defaultdict

In [284]:
merged_data = "Data/merged.csv"
merged_df = pd.read_csv(merged_data)
merged_df

,id,title,image,region,weightWatcherSmartPoints,readyInMinutes,sourceUrl,pricePerServing,healthScore,aggregateLikes,...,glutenFree,dairyFree,servings,instructions,summary,ingredients,calories,fat,carbs,protein
0,989129,Chicken Curry Fried Rice,https://spoonacular.com/recipeImages/989129-31...,chinese,14,20,http://jeanetteshealthyliving.com/chicken-curr...,143.80,72.0,2041,...,True,True,2,InstructionsPlace carrots and water to just co...,Chicken Curry Fried Rice takes approximately <...,1/4 cup toasted sliced almonds; 1/2 cup carrot...,568,23g,64g,29g
1,536175,Quinoa Veggie “Fried Rice”,https://spoonacular.com/recipeImages/536175-31...,chinese,9,30,http://DAMNDELICIOUS.NET/post/33882952382/quin...,185.86,58.0,9911,...,True,True,4,Heat 1 tablespoon olive oil in a medium skille...,The recipe Quinoa Veggie “Fried Rice” could sa...,"1 head broccoli, cut into florets; 2 carrots, ...",409,14g,57g,20g
2,577707,Shaved Brussels Sprouts and Ginger Potstickers,https://spoonacular.com/recipeImages/577707-31...,chinese,16,30,http://naturallyella.com/2013/10/15/shaved-bru...,304.86,63.0,4298,...,False,True,1,Heat sesame oil in a large skillet with a lid....,The recipe Shaved Brussels Sprouts and Ginger ...,1¼ cup shredded brussels sprouts; 2 tablespoon...,592,28g,73g,17g
3,757011,Egg Roll Stir-Fry (aka Egg Roll in a Bowl),https://spoonacular.com/recipeImages/757011-31...,chinese,11,25,http://www.heydonna.com/2015/03/egg-roll-stir-...,150.26,58.0,861,...,True,True,4,"Mix garlic, ginger and soy sauce in a small bo...",Egg Roll Stir-Fry (akan Egg Roll in a Bowl) mi...,1 medium head of cabbage sliced thin (aprox. 4...,423,28g,21g,24g
4,630113,Chicken chow mein,https://spoonacular.com/recipeImages/630113-31...,chinese,14,40,http://www.bbc.co.uk/food/recipes/chickenchowm...,311.62,58.0,1067,...,False,True,2,<p>Cook the noodles in a pan of boiling water ...,Chicken chow mein might be just the <b>Chinese...,1 tsp five-spice powder; 150g 5oz bean sprouts...,624,15g,73g,49g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,711188,Korean Steak and Noodle Lettuce Cups,https://spoonacular.com/recipeImages/711188-31...,korean,12,30,http://www.runningtothekitchen.com/korean-stea...,310.04,100.0,72,...,False,True,4,Whisk all the ingredients together for the sau...,The recipe Korean Steak and Noodle Lettuce Cup...,"1 head bibb lettuce, outer leaves carefully re...",459,14g,52g,33g
99,535177,Slow Cooker Korean BBQ Pot Roast and Sweet Pot...,https://spoonacular.com/recipeImages/535177-31...,korean,5,605,http://sarahscucinabella.com/2014/04/17/slow-c...,168.26,80.0,132,...,False,True,6,Layer the onions and sweet potatoes in the bot...,The recipe Slow Cooker Korean BBQ Pot Roast an...,2-3 lb beef eye round roast; 1 pouch Campbells...,268,8g,13g,35g
100,202359,"Squash, Shiitake, Kale, and Kimchi Stew",https://spoonacular.com/recipeImages/202359-31...,korean,7,30,http://www.seriouseats.com/recipes/2013/09/din...,788.37,79.0,183,...,True,True,4,Procedures ...,"Squash, Shiitake, Kale, and Kimchi Stew might ...","4 medium garlic cloves, minced (about 4 teaspo...",296,15g,27g,20g
101,594616,Slow Cooker Korean Tacos,https://spoonacular.com/recipeImages/594616-31...,korean,12,550,http://www.tablefortwoblog.com/slow-cooker-kor...,320.29,55.0,1932,...,True,True,6,Add the pieces of pork in the insert/bowl of y...,The recipe Slow Cooker Korean Tacos is ready <...,"2 pounds boneless pork loin, cut into 4 pieces...",436,10g,45g,40g


In [285]:
df = merged_df[['title', 'region', 'ingredients']]

In [286]:
region = df.drop_duplicates('region')
region_list = []
for i in range(len(region['region'])):
    region_list.append(region.iloc[i,1])
region_list

['chinese',
 'indian',
 'french',
 'italian',
 'mexican',
 'southern',
 'greek',
 'nordic',
 'caribbean',
 'middle eastern',
 'korean']

In [306]:
#append region to food_dict
food_dict = {}
food_dict['name'] = 'flare'
food_dict['children'] = []

for i in range(len(region_list)):
    region_dict = {}
    region_dict['name'] = f"{region_list[i]}"
    region_dict['children'] = []
    food_dict['children'].append(region_dict)
    for x in range(len(df)):
        region_df = df[df['region'] == region_list[i]]
    list = []
    for row in region_df['ingredients']:
        splited=row.split('; ')
        list.append(splited)
    for y in range(len(region_df)):
        dish_dict = {}
        dish_dict['name'] = f"{region_df.iloc[y,0]}"
        dish_dict['children'] = []
        region_dict['children'].append(dish_dict)
        ingredient_list = list[y]
        for z in range(len(ingredient_list)):
            ingredient_dict = {}
            ingredient_dict['name'] = ingredient_list[z]
            ingredient_dict['value'] = 1
            dish_dict['children'].append(ingredient_dict)


In [307]:
food_dict['children'][10]

{'name': 'korean',
 'children': [{'name': 'Korean Spicy BBQ Chicken Tacos',
   'children': [{'name': '1 cup onion and cilantro relish', 'value': 1},
    {'name': '1 cup Korean style salsa roja', 'value': 1},
    {'name': '2 cups shredded lettuce in a Korean sesame vinaigrette',
     'value': 1},
    {'name': '6 tablespoons sesame seeds (toasted and crushed)', 'value': 1},
    {'name': '12 small tortillas', 'value': 1}]},
  {'name': 'Korean Short Rib Tacos',
   'children': [{'name': '1 cup onion and cilantro relish', 'value': 1},
    {'name': '1 cup Korean style salsa roja', 'value': 1},
    {'name': '2 cups shredded lettuce in a Korean sesame vinaigrette',
     'value': 1},
    {'name': '6 tablespoons sesame seeds (toasted and crushed)', 'value': 1},
    {'name': '12 small tortillas', 'value': 1}]},
  {'name': 'Cauliflower Rice Kimchi Bowls',
   'children': [{'name': '1 avocado, diced', 'value': 1},
    {'name': '1 small head of cauliflower, riced', 'value': 1},
    {'name': '1/3 cup c

In [308]:
list

[['1 cup onion and cilantro relish',
  '1 cup Korean style salsa roja',
  '2 cups shredded lettuce in a Korean sesame vinaigrette',
  '6 tablespoons sesame seeds (toasted and crushed)',
  '12 small tortillas'],
 ['1 cup onion and cilantro relish',
  '1 cup Korean style salsa roja',
  '2 cups shredded lettuce in a Korean sesame vinaigrette',
  '6 tablespoons sesame seeds (toasted and crushed)',
  '12 small tortillas'],
 ['1 avocado, diced',
  '1 small head of cauliflower, riced',
  '1/3 cup coconut milk',
  '½ clove garlic, minced',
  '1 teaspoon minced ginger',
  '6 leaves of curly kale, stemmed and torn',
  "½ cup kimchi (I used Mother in Law's kimchi)",
  'lime slices, for serving',
  '1 tablespoon rice vinegar or fresh lime juice',
  '¼ cup microgreens, optional',
  '2 tablespoons white miso paste',
  'extra-virgin olive oil, for drizzling',
  '½ teaspoon rice vinegar',
  '½ cup chopped scallions',
  'pinch of sea salt',
  'sea salt',
  'sprinkle of sesame seeds, optional',
  '7 oun

In [206]:
# food_dict = {'name': 'flare',
#              'children': [{'name': 'chinese',
#                           'children': [{'name':'dish1',
#                                         'children': [{'name': "ingredient1", 'value': 1},
#                                                      {'name': "ingredient2", 'value': 1},
#                                                      {'name': "ingredient3", 'value': 1}
#                                                     ]
#                                        },
#                                       {'name':'dish2',
#                                         'children': [{'name': "ingredient1", 'value': 1},
#                                                      {'name': "ingredient2", 'value': 1},
#                                                      {'name': "ingredient3", 'value': 1}
#                                                     ]}
#                                        ]},
#                             {'name': 'italian',
#                               'children': [{'name':'dish1',
#                                         'children': [{'name': "ingredient1", 'value': 1},
#                                                      {'name': "ingredient2", 'value': 1},
#                                                      {'name': "ingredient3", 'value': 1}
#                                                     ]
#                                        },
#                                       {'name':'dish2',
#                                         'children': [{'name': "ingredient1", 'value': 1},
#                                                      {'name': "ingredient2", 'value': 1},
#                                                      {'name': "ingredient3", 'value': 1}
#                                                     ]}
#                                        ]}
#                           ]
# }

In [53]:
# food_json = json.dumps(food_dict)
# food_json

'{"name": "flare", "children": [{"name": "chinese", "children": [{"name": "dish1", "children": [{"name": "ingredient1", "value": 1}, {"name": "ingredient2", "value": 1}, {"name": "ingredient3", "value": 1}]}, {"name": "dish2", "children": [{"name": "ingredient1", "value": 1}, {"name": "ingredient2", "value": 1}, {"name": "ingredient3", "value": 1}]}]}, {"name": "italian", "children": [{"name": "dish1", "children": [{"name": "ingredient1", "value": 1}, {"name": "ingredient2", "value": 1}, {"name": "ingredient3", "value": 1}]}, {"name": "dish2", "children": [{"name": "ingredient1", "value": 1}, {"name": "ingredient2", "value": 1}, {"name": "ingredient3", "value": 1}]}]}]}'

In [309]:
with open('Data/food.json', 'w') as json_file:
  json.dump(food_dict, json_file)

In [8]:
# with open('Data/flare-2.json') as f:
#   data = json.load(f)

In [130]:
# data['children'][0]

{'name': 'analytics',
 'children': [{'name': 'cluster',
   'children': [{'name': 'AgglomerativeCluster', 'value': 3938},
    {'name': 'CommunityStructure', 'value': 3812},
    {'name': 'HierarchicalCluster', 'value': 6714},
    {'name': 'MergeEdge', 'value': 743}]},
  {'name': 'graph',
   'children': [{'name': 'BetweennessCentrality', 'value': 3534},
    {'name': 'LinkDistance', 'value': 5731},
    {'name': 'MaxFlowMinCut', 'value': 7840},
    {'name': 'ShortestPaths', 'value': 5914},
    {'name': 'SpanningTree', 'value': 3416}]},
  {'name': 'optimization',
   'children': [{'name': 'AspectRatioBanker', 'value': 7074}]}]}